In [1]:
import sys
import os
sys.path.append("/home/smets/codes/far/PHARE/pyphare/")
import pyphare
import matplotlib.pyplot as plt
from pyphare.pharesee.run import Run
import numpy as np
%matplotlib widget

In [2]:
run_path = "/DATA/phare/harris/tiny"
os.listdir(run_path)

['EM_B.h5',
 'EM_E.h5',
 'harris-688134.log',
 'harris.py',
 'ions_bulkVelocity.h5',
 'ions_density.h5',
 'ions_pop_protons_density.h5',
 'ions_pop_protons_domain.h5',
 'ions_pop_protons_flux.h5',
 'job.sh']

In [3]:
run  = Run(run_path)
time = 7.0
B = run.GetB(time)
h5_time_grp_key = 't'

for ilvl, lvl in B.levels(time).items():
    for patch in lvl.patches:
        print(patch.layout.box.shape)
        break

[45 45]


In [4]:
from pyphare.pharesee.hierarchy import hierarchy_fromh5
from pyphare.pharesee.hierarchy import hierarchy_toh5
import h5py

h5file = os.path.join(run_path, 'EM_B.h5')
f = h5py.File(h5file)
#f.visit(print)
C = hierarchy_fromh5(h5file, "{0:.10f}".format(time), None, silent=False)

#f.visit(print)
# print("f : ", list(f.attrs.keys()))
# print("f : ", list(f.keys()))
# print("f[t] :", list(f['t'].attrs.keys()))
# print("f[t] : ", list(f['t'].keys()))
# print("f[t][0.0]", list(f['t']['0.0000000000'].attrs.keys()))
# print("f[t][0.0]", list(f['t']['0.0000000000'].keys()))
# print("f[t][0.0][pl0] : ", list(f['t']['0.0000000000']['pl0'].attrs.keys()))
# print("f[t][0.0][pl0] : ", list(f['t']['0.0000000000']['pl0'].keys()))
# print("f[t]{0.0][pl0]{p0#0] : ", list(f['t']['0.0000000000']['pl0']['p0#0'].attrs.keys()))
# print("f[t]{0.0][pl0]{p0#0] : ", list(f['t']['0.0000000000']['pl0']['p0#0'].keys()))
# print("f[t]{0.0][pl0]{p0#0][EM_B_x] : ", list(f['t']['0.0000000000']['pl0']['p0#0']['EM_B_x'].attrs.keys()))
# print(list(f ['t']['0.0000000000']['pl0']['p0#0']['EM_B_x']))
# print("f[t]{0.0][pl0]{p0#0][EM_B_x] : ", type(f['t']['0.0000000000']['pl0']['p0#0']['EM_B_x']))
# print("f[t]{0.0][pl0]{p0#0][EM_B_x] : ", dir(f['t']['0.0000000000']['pl0']['p0#0']['EM_B_x']))
# print("f[t]{0.0][pl0]{p0#0][EM_B_x] : ", f['t']['0.0000000000']['pl0']['p0#0']['EM_B_x'].dtype)
# print("f[t]{0.0][pl0]{p0#0][EM_B_x] : ", f['t']['0.0000000000']['pl0']['p0#0']['EM_B_x'].shape)

for patch in list(f['t']['0.0000000000']['pl0'].keys()):
    print(patch, " : ", f['t']['0.0000000000']['pl0'][patch]['EM_B_x'].shape)


f.close()

creating hierarchy from time 7.0000000000
p0#0  :  (50, 49)
p1#1  :  (50, 48)
p10#12  :  (50, 48)
p11#13  :  (45, 54)
p12#14  :  (45, 54)
p13#15  :  (45, 54)
p14#16  :  (45, 54)
p15#17  :  (50, 48)
p16#18  :  (43, 57)
p17#19  :  (42, 57)
p18#20  :  (38, 51)
p18#21  :  (50, 16)
p19#22  :  (47, 51)
p2#2  :  (60, 32)
p2#3  :  (50, 15)
p3#4  :  (60, 40)
p4#5  :  (60, 40)
p5#6  :  (50, 48)
p6#7  :  (41, 60)
p7#8  :  (40, 60)
p8#10  :  (15, 48)
p8#9  :  (34, 60)
p9#11  :  (50, 48)


In [ ]:
def hierarchy_toh5(h5_filename, time, hier, silent=True):
    import h5py
    h5file = os.path.join(run_path, h5_filename)

    if os.path.isfile(h5file):
        f = h5py.File(h5file, 'r+')
    else:
        f = h5py.File(h5file, 'w-')

        f.attrs['cell_width'] = hier.patch_levels[0].patches[0].dl
        f.attrs['dimension'] = hier.ndim
        f.attrs['domain_box'] = hier.domain_box.upper-hier.domain_box.lower
        f.attrs['interpOrder'] = hier.patch_levels[0].patches[0].layout.interp_order
        f.attrs['layoutType'] = hier.patch_levels[0].patches[0].layout.impl
        f.attrs['origin'] = hier.patch_levels[0].patches[0].layout.origin

    if 't' not in f:
        base_node = f.create_group(h5_time_grp_key)
    else:
        base_node = f.get('t')

    print(list(base_node))

    if "{0:.10f}".format(time) not in list(base_node):
        time_node = base_node.create_group("{0:.10f}".format(time))

        for ilvl, lvl in hier.levels(time).items():
            level_node = time_node.create_group("pl{0:d}".format(ilvl))
            for patch in lvl.patches:
                patch_node = level_node.create_group(patch.id)

                patch_node.attrs['origin'] = patch.origin
                patch_node.attrs['lower'] = patch.box.lower
                patch_node.attrs['upper'] = patch.box.upper
                patch_node.attrs['mpi_rank'] = patch.id.strip('p').split('#')[0]
                patch_node.attrs['nbrCells'] = patch.layout.box.shape

                # print(patch.id, " : ", patch.patch_datas['Bx'].dataset.shape)
                patch_node.create_dataset('EM_B_x', data=patch.patch_datas['Bx'].dataset, dtype='float64')

    else:
        time_node = base_node.get("{0:.10f}".format(time))
        print("group 't/{}' already exist, so do the associated dataset : no need to write in h5 file".format(time))

    f.close()

In [5]:
zob_file = os.path.join(run_path, "zob.h5")

if os.path.isfile(zob_file):
    # os.remove(zob_file)
    pass

hierarchy_toh5(zob_file, time, B)

In [6]:
h5file = os.path.join(run_path, "zob.h5")

# line 1153, we have : data_file = h5py.File(h5_filename, "r")
# which should be closed... before the "return" of line 1170, 1205, 1245, 1273, 1281

D = hierarchy_fromh5(h5file, "{0:.10f}".format(time), None, silent=False)

creating hierarchy from time 7.0000000000


In [7]:
print(D)

Hierarchy: 
Level 0
    P0 <class 'h5py._hl.dataset.Dataset'> Bx box is Box([0, 0],[44, 44]) and ghost box is Box([-2, -2],[46, 46])
    P1 <class 'h5py._hl.dataset.Dataset'> Bx box is Box([0, 56],[44, 99]) and ghost box is Box([-2, 54],[46, 101])
    P2 <class 'h5py._hl.dataset.Dataset'> Bx box is Box([200, 0],[244, 43]) and ghost box is Box([198, -2],[246, 45])
    P3 <class 'h5py._hl.dataset.Dataset'> Bx box is Box([320, 50],[359, 99]) and ghost box is Box([318, 48],[361, 101])
    P4 <class 'h5py._hl.dataset.Dataset'> Bx box is Box([360, 50],[399, 99]) and ghost box is Box([358, 48],[401, 101])
    P5 <class 'h5py._hl.dataset.Dataset'> Bx box is Box([360, 0],[399, 49]) and ghost box is Box([358, -2],[401, 51])
    P6 <class 'h5py._hl.dataset.Dataset'> Bx box is Box([320, 0],[359, 49]) and ghost box is Box([318, -2],[361, 51])
    P7 <class 'h5py._hl.dataset.Dataset'> Bx box is Box([200, 56],[244, 99]) and ghost box is Box([198, 54],[246, 101])
    P8 <class 'h5py._hl.dataset.Datase